In [1]:
import pandas as pd
import torch

In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [4]:
print(train_data.iloc[0:4, [0,1,2,3,-3,-2,-1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [5]:
print(train_data.loc[0:4, 'MSSubClass'])

0    60
1    20
2    60
3    70
4    60
Name: MSSubClass, dtype: int64


In [6]:
train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
train_data.iloc[1:2, 1:-1]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal


In [8]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

In [9]:
all_features.shape

(2919, 79)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
# scaler = StandardScaler()
# all_features.fillna(all_features.mean(), inplace=True)
# all_features = pd.DataFrame(scaler.fit_transform(all_features))

In [12]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
# scaler = StandardScaler()
# all_features[numeric_features].fillna(all_features[numeric_features].mean(), inplace=True)
# all_features[numeric_features] = pd.DataFrame(scaler.fit_transform(all_features[numeric_features]))

In [13]:
all_features[numeric_features] = all_features[numeric_features].apply(lambda x:(x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [15]:
all_features = pd.get_dummies(all_features, dummy_na=True, dtype=int)

In [16]:
all_features.shape

(2919, 330)

In [8]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

In [9]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [10]:
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True, dtype = int)
all_features.shape

(2919, 330)

In [17]:
%matplotlib inline
import numpy as np
from torch import nn

In [20]:
import torch

以下代码报错
can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.


In [18]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1,1), dtype=torch.float32)

In [13]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(
    train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [14]:
all_features.shape

(2919, 330)

In [19]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

In [20]:
def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

In [21]:
from torch.utils import data

In [22]:
def load_array(dat_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*dat_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [23]:
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = load_array((train_features, train_labels), batch_size)
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay = weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls
    

K折交叉验证

In [25]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j==i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [27]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        # if i == 0:
            # d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
            #          xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
            #          legend=['train', 'valid'], yscale='log')
        print(f'折{i + 1}，训练log rmse{float(train_ls[-1]):f}, '
              f'验证log rmse{float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [28]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

折1，训练log rmse0.169762, 验证log rmse0.156616
折2，训练log rmse0.162879, 验证log rmse0.192258
折3，训练log rmse0.163654, 验证log rmse0.168077
折4，训练log rmse0.168113, 验证log rmse0.154437
折5，训练log rmse0.163545, 验证log rmse0.182882
5-折验证: 平均训练log rmse: 0.165591, 平均验证log rmse: 0.170854


In [29]:
def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1024),
                        nn.ReLU(),
                        nn.Linear(1024, 1))
    return net

In [30]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 0.1, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

折1，训练log rmse0.057473, 验证log rmse0.122861
折2，训练log rmse0.052107, 验证log rmse0.145613
折3，训练log rmse0.049253, 验证log rmse0.148260
折4，训练log rmse0.048281, 验证log rmse0.116640
折5，训练log rmse0.044682, 验证log rmse0.164712
5-折验证: 平均训练log rmse: 0.050359, 平均验证log rmse: 0.139617


In [31]:
k, num_epochs, lr, weight_decay, batch_size = 5, 500, 0.1, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

折1，训练log rmse0.005286, 验证log rmse0.137493
折2，训练log rmse0.006173, 验证log rmse0.172997
折3，训练log rmse0.008152, 验证log rmse0.163754
折4，训练log rmse0.003865, 验证log rmse0.138263
折5，训练log rmse0.003681, 验证log rmse0.171953
5-折验证: 平均训练log rmse: 0.005432, 平均验证log rmse: 0.156892


In [32]:
k, num_epochs, lr, weight_decay, batch_size = 7, 200, 0.1, 0.001, 256
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

折1，训练log rmse0.046667, 验证log rmse0.140363
折2，训练log rmse0.067924, 验证log rmse0.130582
折3，训练log rmse0.074562, 验证log rmse0.150231
折4，训练log rmse0.077466, 验证log rmse0.159089
折5，训练log rmse0.057692, 验证log rmse0.128941
折6，训练log rmse0.047928, 验证log rmse0.144576
折7，训练log rmse0.067902, 验证log rmse0.173693
7-折验证: 平均训练log rmse: 0.062877, 平均验证log rmse: 0.146782


In [ ]:
from sklearn.model_selection import KFold

train_iter = data.DataLoader((train_features, train_labels), batch_size)
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_data, train_label in kf.split(train_iter):
    for epoch in range(num_epochs):
        

分隔符

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn

In [2]:
df_train_data = pd.read_csv('./data/train.csv')
df_test_data = pd.read_csv('./data/test.csv')

In [4]:
print(df_train_data.shape)
print(df_test_data.shape)

(1460, 81)
(1459, 80)


In [5]:
print(df_train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [7]:
all_features = pd.concat((df_train_data.iloc[:, 1:-1], df_test_data.iloc[:, 1:]))

In [8]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [9]:
all_features = pd.get_dummies(all_features, dummy_na=True, dtype=int)
all_features.shape

(2919, 330)

通过values属性，将pandas格式中提取出NumPy格式，并将其转换为张量表示用于训练

In [10]:
n_train = df_train_data.shape[0]
X_train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
X_test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
y_train_labels = torch.tensor(df_train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [11]:
loss = nn.MSELoss()
in_features = X_test_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

In [12]:
def los_rmse(net, features,  labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

In [13]:
def load_array(dat_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*dat_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [14]:
def train(net, X_train_features, y_train_labels, X_test_features, y_test_labels, num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = load_array((X_train_features, y_train_labels), batch_size)
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l  = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, X_train_features, y_train_labels))
        if y_test_labels is not None:
            test_ls.append(log_rmse(net, X_test_features, y_test_labels))
    return train_ls, test_ls

In [20]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [21]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        # if i == 0:
            # d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
            #          xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
            #          legend=['train', 'valid'], yscale='log')
        print(f'折{i + 1}，训练log rmse{float(train_ls[-1]):f}, '
              f'验证log rmse{float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [22]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, X_train_features, y_train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

NameError: name 'data' is not defined